In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130600

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,61230.9845
2018-02-28,71520.8297
2018-03-31,68813.9829
2018-04-30,71727.4463
2018-05-31,67620.3940
2018-06-30,70960.3548
2018-07-31,74484.5294
2018-08-31,103328.7805
2018-09-30,86772.3410


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31     61230.9845
 2018-02-28     71520.8297
 2018-03-31     68813.9829
 2018-04-30     71727.4463
 2018-05-31     67620.3940
 2018-06-30     70960.3548
 2018-07-31     74484.5294
 2018-08-31    103328.7805
 2018-09-30     86772.3410
 2018-10-31     85694.3140
 2018-11-30     76487.5470
 2018-12-31     74136.4325
 2019-01-31    101050.6216
 2019-02-28     77143.9979
 2019-03-31     81787.5359
 2019-04-30     89497.8631
 2019-05-31     92341.3296
 2019-06-30     89753.9974
 2019-07-31     85678.0041
 2019-08-31     90350.4787
 2019-09-30     72920.9447
 2019-10-31     88767.5396
 2019-11-30     86581.5434
 2019-12-31     71940.9695
 2020-01-31     75657.5383
 2020-02-29     77090.3406
 2020-03-31     72275.4748
 2020-04-30     57056.0192
 2020-05-31     62448.0177
 2020-06-30     66410.0819
 2020-07-31     90787.4802
 2020-08-31     83713.5639
 2020-09-30     79463.9654
 2020-10-31    130241.3675
 2020-11-30    171862.7318
 2020-12-31    199667.8574

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -2.426958
p-value : 0.134272
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -3.154979
p-value : 0.022743
Critical Values :
	1%: -3.632743
	5%: -2.948510
	10%: -2.613017


In [9]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,1,0)(0,0,0)[1] intercept   : AIC=935.422, Time=0.01 sec
 ARIMA(0,1,1)(0,0,0)[1] intercept   : AIC=937.492, Time=0.07 sec
 ARIMA(0,1,2)(0,0,0)[1] intercept   : AIC=937.284, Time=0.03 sec
 ARIMA(0,1,3)(0,0,0)[1] intercept   : AIC=938.455, Time=0.06 sec
 ARIMA(0,1,4)(0,0,0)[1] intercept   : AIC=938.278, Time=0.10 sec
 ARIMA(0,1,5)(0,0,0)[1] intercept   : AIC=942.433, Time=0.08 sec
 ARIMA(1,1,0)(0,0,0)[1] intercept   : AIC=937.385, Time=0.03 sec
 ARIMA(1,1,1)(0,0,0)[1] intercept   : AIC=938.597, Time=0.07 sec
 ARIMA(1,1,2)(0,0,0)[1] intercept   : AIC=939.224, Time=0.11 sec
 ARIMA(1,1,3)(0,0,0)[1] intercept   : AIC=938.980, Time=0.12 sec
 ARIMA(1,1,4)(0,0,0)[1] intercept   : AIC=940.280, Time=0.22 sec
 ARIMA(2,1,0)(0,0,0)[1] intercept   : AIC=937.398, Time=0.05 sec
 ARIMA(2,1,1)(0,0,0)[1] intercept   : AIC=939.164, Time=0.09 sec
 ARIMA(2,1,2)(0,0,0)[1] intercept   : AIC=941.053, Time=0.16 sec
 ARIMA(2,1,3)(0,0,0)[1] intercept   : AIC=939.068, Time=0.26 sec
 ARIMA(3,1,0)(0,0,0)[1] i

ARIMA(order=(0, 1, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [10]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(0,1,0))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                             ARIMA Model Results                              
Dep. Variable:                    D.y   No. Observations:                   41
Model:                 ARIMA(0, 1, 0)   Log Likelihood                -465.711
Method:                           css   S.D. of innovations          20741.050
Date:                Sat, 11 Sep 2021   AIC                            935.422
Time:                        04:46:07   BIC                            938.849
Sample:                             1   HQIC                           936.670
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       1087.3043   3239.208      0.336      0.737   -5261.426    7436.035


C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [11]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([106897.76704634, 107985.07139269]), array([20741.05001577, 29332.27423016]), array([[ 66246.05601389, 147549.4780788 ],
       [ 50494.87031692, 165475.27246845]]))
